In [63]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 15: Warehouse Woes ---</h2><p>You appear back inside your own mini submarine! Each Historian drives their mini submarine in a different direction; maybe the Chief has his own submarine down here somewhere as well?</p>
<p>You look up to see a vast school of <a href="/2021/day/6">lanternfish</a> swimming past you. On closer inspection, they seem quite anxious, so you drive your mini submarine over to see if you can help.</p>
<p>Because lanternfish populations grow rapidly, they need a lot of food, and that food needs to be stored somewhere. That's why these lanternfish have built elaborate warehouse complexes operated by robots!</p>
<p>These lanternfish seem so anxious because they have lost control of the robot that operates one of their most important warehouses! It is currently running <span title="Wesnoth players might solve their Warehouse Woes with a Warehouse Wose!">amok</span>, pushing around boxes in the warehouse with no regard for lanternfish logistics <em>or</em> lanternfish inventory management strategies.</p>
<p>Right now, none of the lanternfish are brave enough to swim up to an unpredictable robot so they could shut it off. However, if you could anticipate the robot's movements, maybe they could find a safe option.</p>
<p>The lanternfish already have a map of the warehouse and a list of movements the robot will <em>attempt</em> to make (your puzzle input). The problem is that the movements will sometimes fail as boxes are shifted around, making the actual movements of the robot difficult to predict.</p>
<p>For example:</p>
<pre><code>##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

&lt;vv&gt;^&lt;v^&gt;v&gt;^vv^v&gt;v&lt;&gt;v^v&lt;v&lt;^vv&lt;&lt;&lt;^&gt;&lt;&lt;&gt;&lt;&gt;&gt;v&lt;vvv&lt;&gt;^v^&gt;^&lt;&lt;&lt;&gt;&lt;&lt;v&lt;&lt;&lt;v^vv^v&gt;^
vvv&lt;&lt;^&gt;^v^^&gt;&lt;&lt;&gt;&gt;&gt;&lt;&gt;^&lt;&lt;&gt;&lt;^vv^^&lt;&gt;vvv&lt;&gt;&gt;&lt;^^v&gt;^&gt;vv&lt;&gt;v&lt;&lt;&lt;&lt;v&lt;^v&gt;^&lt;^^&gt;&gt;&gt;^&lt;v&lt;v
&gt;&lt;&gt;vv&gt;v^v^&lt;&gt;&gt;&lt;&gt;&gt;&gt;&gt;&lt;^^&gt;vv&gt;v&lt;^^^&gt;&gt;v^v^&lt;^^&gt;v^^&gt;v^&lt;^v&gt;v&lt;&gt;&gt;v^v^&lt;v&gt;v^^&lt;^^vv&lt;
&lt;&lt;v&lt;^&gt;&gt;^^^^&gt;&gt;&gt;v^&lt;&gt;vvv^&gt;&lt;v&lt;&lt;&lt;&gt;^^^vv^&lt;vvv&gt;^&gt;v&lt;^^^^v&lt;&gt;^&gt;vvvv&gt;&lt;&gt;&gt;v^&lt;&lt;^^^^^
^&gt;&lt;^&gt;&lt;&gt;&gt;&gt;&lt;&gt;^^&lt;&lt;^^v&gt;&gt;&gt;&lt;^&lt;v&gt;^&lt;vv&gt;&gt;v&gt;&gt;&gt;^v&gt;&lt;&gt;^v&gt;&lt;&lt;&lt;&lt;v&gt;&gt;v&lt;v&lt;v&gt;vvv&gt;^&lt;&gt;&lt;&lt;&gt;^&gt;&lt;
^&gt;&gt;&lt;&gt;^v&lt;&gt;&lt;^vvv&lt;^^&lt;&gt;&lt;v&lt;&lt;&lt;&lt;&lt;&gt;&lt;^v&lt;&lt;&lt;&gt;&lt;&lt;&lt;^^&lt;v&lt;^^^&gt;&lt;^&gt;&gt;^&lt;v^&gt;&lt;&lt;&lt;^&gt;&gt;^v&lt;v^v&lt;v^
&gt;^&gt;&gt;^v&gt;vv&gt;^&lt;&lt;^v&lt;&gt;&gt;&lt;&lt;&gt;&lt;&lt;v&lt;&lt;v&gt;&lt;&gt;v&lt;^vv&lt;&lt;&lt;&gt;^^v^&gt;^^&gt;&gt;&gt;&lt;&lt;^v&gt;&gt;v^v&gt;&lt;^^&gt;&gt;^&lt;&gt;vv^
&lt;&gt;&lt;^^&gt;^^^&lt;&gt;&lt;vvvvv^v&lt;v&lt;&lt;&gt;^v&lt;v&gt;v&lt;&lt;^&gt;&lt;&lt;&gt;&lt;&lt;&gt;&lt;&lt;&lt;^^&lt;&lt;&lt;^&lt;&lt;&gt;&gt;&lt;&lt;&gt;&lt;^^^&gt;^^&lt;&gt;^&gt;v&lt;&gt;
^^&gt;vv&lt;^v^v&lt;vv&gt;^&lt;&gt;&lt;v&lt;^v&gt;^^^&gt;&gt;&gt;^^vvv^&gt;vvv&lt;&gt;&gt;&gt;^&lt;^&gt;&gt;&gt;&gt;&gt;^&lt;&lt;^v&gt;^vvv&lt;&gt;^&lt;&gt;&lt;&lt;v&gt;
v^^&gt;&gt;&gt;&lt;&lt;^^&lt;&gt;&gt;^v^&lt;v^vv&lt;&gt;v^&lt;&lt;&gt;^&lt;^v^v&gt;&lt;^&lt;&lt;&lt;&gt;&lt;&lt;^&lt;v&gt;&lt;v&lt;&gt;vv&gt;&gt;v&gt;&lt;v^&lt;vv&lt;&gt;v^&lt;&lt;^
</code></pre>

<p>As the robot (<code>@</code>) attempts to move, if there are any boxes (<code>O</code>) in the way, the robot will also attempt to push those boxes. However, if this action would cause the robot or a box to move into a wall (<code>#</code>), nothing moves instead, including the robot. The initial positions of these are shown on the map at the top of the document the lanternfish gave you.</p>
<p>The rest of the document describes the <em>moves</em> (<code>^</code> for up, <code>v</code> for down, <code>&lt;</code> for left, <code>&gt;</code> for right) that the robot will attempt to make, in order. (The moves form a single giant sequence; they are broken into multiple lines just to make copy-pasting easier. Newlines within the move sequence should be ignored.)</p>
<p>Here is a smaller example to get started:</p>
<pre><code>########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########
          
&lt;^^&gt;&gt;&gt;vv&lt;v&gt;&gt;v&lt;&lt;
</code></pre>

<p>Were the robot to attempt the given sequence of moves, it would push around the boxes as follows:</p>
<pre><code>Initial state:
########
#..O.O.#
##<em>@</em>.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########<br/>
Move &lt;:<br/>
########
#..O.O.# 
##<em>@</em>.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########<br/>
Move ^:
########
#.<em>@</em>O.O.#
##..O..#
#...O..#
#.#.O..#
#...O..#
#......#
########<br/>
Move ^:
########
#.<em>@</em>O.O.#
##..O..#
#...O..#
#.#.O..#
#...O..#
#......#
########<br/>
Move &gt;:
########
#..<em>@</em>OO.#
##..O..#
#...O..#
#.#.O..#
#...O..#
#......#
########<br/>
Move &gt;:
########
#...<em>@</em>OO#
##..O..#
#...O..#
#.#.O..#
#...O..#
#......#
########<br/>
Move &gt;:
########
#...<em>@</em>OO#
##..O..#
#...O..#
#.#.O..#
#...O..#
#......#
########<br/>
Move v:
########
#....OO#
##..<em>@</em>..#
#...O..#
#.#.O..#
#...O..#
#...O..#
########<br/>
Move v:
########
#....OO#
##..<em>@</em>..#
#...O..#
#.#.O..#
#...O..#
#...O..#
########<br/>
Move &lt;:
########
#....OO#
##.<em>@</em>...#
#...O..#
#.#.O..#
#...O..#
#...O..#
########<br/>
Move v:
########
#....OO#
##.....#
#..<em>@</em>O..#
#.#.O..#
#...O..#
#...O..#
########<br/>
Move &gt;:
########
#....OO#
##.....#
#...<em>@</em>O.#
#.#.O..#
#...O..#
#...O..#
########<br/>
Move &gt;:
########
#....OO#
##.....#
#....<em>@</em>O#
#.#.O..#
#...O..#
#...O..#
########<br/>
Move v:
########
#....OO#
##.....#
#.....O#
#.#.O<em>@</em>.#
#...O..#
#...O..#
########<br/>
Move &lt;:
########
#....OO#
##.....#
#.....O#
#.#O<em>@</em>..#
#...O..#
#...O..#
########<br/>
Move &lt;:
########
#....OO#
##.....#
#.....O#
#.#O<em>@</em>..#
#...O..#
#...O..#
########
</code></pre>

<p>The larger example has many more moves; after the robot has finished those moves, the warehouse would look like this:</p>
<pre><code>##########
#.O.O.OOO#
#........#
#OO......#
#OO<em>@</em>.....#
#O#.....O#
#O.....OO#
#O.....OO#
#OO....OO#
##########
</code></pre>
<p>The lanternfish use their own custom Goods Positioning System (GPS for short) to track the locations of the boxes. The <em>GPS coordinate</em> of a box is equal to 100 times its distance from the top edge of the map plus its distance from the left edge of the map. (This process does not stop at wall tiles; measure all the way to the edges of the map.)</p>
<p>So, the box shown below has a distance of <code>1</code> from the top edge of the map and <code>4</code> from the left edge of the map, resulting in a GPS coordinate of <code>100 * 1 + 4 = 104</code>.</p>
<pre><code>#######
#...O..
#......
</code></pre>
<p>The lanternfish would like to know the <em>sum of all boxes' GPS coordinates</em> after the robot finishes moving. In the larger example, the sum of all boxes' GPS coordinates is <code><em>10092</em></code>. In the smaller example, the sum is <code><em>2028</em></code>.</p>
<p>Predict the motion of the robot and boxes in the warehouse. After the robot is finished moving, <em>what is the sum of all boxes' GPS coordinates?</em></p>
</article>


In [64]:
from more_itertools import first


tests = [
    {
        "name": "Small Example",
        "s": """
                ########
                #..O.O.#
                ##@.O..#
                #...O..#
                #.#.O..#
                #...O..#
                #......#
                ########

                <^^>>>vv<v>>v<<
        """,
        "expected": 2028,
    },
    {
        "name": "Example",
        "s": """
                ##########
                #..O..O.O#
                #......O.#
                #.OO..O.O#
                #..O@..O.#
                #O#..O...#
                #O..O..O.#
                #.OO.O.OO#
                #....O...#
                ##########

                <vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
                vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
                ><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
                <<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
                ^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
                ^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
                >^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
                <><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
                ^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
                v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^
        """,
        "expected": 10092,
    },
]


class WareHouse(Str):
    DIRS = {"^": (-1, 0), ">": (0, 1), "v": (1, 0), "<": (0, -1)}
    BOX = "O"
    EMPTY = "."
    WALL = "#"
    ROBOT = "@"

    def __init__(self, s: str) -> None:
        self.wharehouse, self.moves = self._parse(s)
        self.robot = self._find_robot()

    def make_moves(self, debug=False) -> None:
        self._debug("Initial state:", debug)
        r, c = self.robot

        for i, move in enumerate(self.moves):
            dr, dc = self.DIRS[move]
            rr, cc = r + dr, c + dc

            if self._is_empty(rr, cc):
                self._swap(r, c, rr, cc)
                r, c = rr, cc
            elif self._is_box(rr, cc):
                r, c = self._move_box(r, c, dr, dc, rr, cc)

            self._debug(f"Move {move} step {i}:", debug)

        self._debug(f"Final:", debug)

    def _is_empty(self, r, c) -> bool:
        return self.wharehouse[r][c] == self.EMPTY

    def _is_box(self, r: int, c: int) -> bool:
        return self.wharehouse[r][c] in self.BOX

    def _is_box_start(self, r: int, c: int) -> bool:
        return self.wharehouse[r][c] == self.BOX[0]

    def _move_box(
        self, r: int, c: int, dr: int, dc: int, rr: int, cc: int
    ) -> tuple[int, int]:
        # determine number of O's and swap back from last to first
        # if first cell after last is empty
        rrr, ccc = rr + dr, cc + dc
        while self._is_box(rrr, ccc):
            rrr, ccc = rrr + dr, ccc + dc

        if self._is_empty(rrr, ccc):
            while (rrr, ccc) != (rr, cc):
                self._swap(rrr, ccc, rrr - dr, ccc - dc)
                rrr, ccc = rrr - dr, ccc - dc

            self._swap(r, c, rr, cc)
            return rr, cc

        return r, c

    def sumGPS(self) -> int:
        rows, cols = len(self.wharehouse), len(self.wharehouse[0])
        return sum(
            100 * r + c
            for r, c in product(range(rows), range(cols))
            if self._is_box_start(r, c)
        )

    def _swap(self, r: int, c: int, rr: int, cc: int) -> None:
        self.wharehouse[rr][cc], self.wharehouse[r][c] = (
            self.wharehouse[r][c],
            self.wharehouse[rr][cc],
        )

    def _find_robot(self) -> tuple[int, int]:
        rows, cols = len(self.wharehouse), len(self.wharehouse[0])
        return first(
            (r, c)
            for r, c in product(range(rows), range(cols))
            if self.wharehouse[r][c] == self.ROBOT
        )

    def _debug(self, header: str, debug: bool) -> None:
        if debug:
            print(header)
            print(self)
            print()

    @classmethod
    def _parse(cls, s) -> tuple[list[list[str]], str]:
        wharehouse, moves = re.split(r"(?:\r?\n){2,}", s.strip())
        wharehouse = [list(l.strip()) for l in wharehouse.splitlines()]
        moves = re.sub(r"\s+", "", moves.strip())
        return wharehouse, moves

    def __str__(self) -> str:
        return "\n".join("".join(l) for l in self.wharehouse)

    @classmethod
    def solve(cls, s: str, debug=False) -> int:
        wh = cls(s)
        wh.make_moves(debug=debug)
        sumGPS = wh.sumGPS()
        return sumGPS


@test(tests=tests[:])
def partI_test(s: str) -> int:
    return WareHouse.solve(s)


Test Small Example passed, for partI_test.
Test Example passed, for partI_test.
Success


In [65]:
with open("../input/day15.txt") as f:
    puzzle = f.read()

print(f"Part I: {WareHouse.solve(puzzle)}")

Part I: 1406392


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1406392</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The lanternfish use your information to find a safe moment to swim in and turn off the malfunctioning robot! Just as they start preparing a festival in your honor, reports start coming in that a <em>second</em> warehouse's robot is <em>also</em> malfunctioning.</p>
<p>This warehouse's layout is surprisingly similar to the one you just helped. There is one key difference: everything except the robot is <em>twice as wide</em>! The robot's list of movements doesn't change.</p>
<p>To get the wider warehouse's map, start with your original map and, for each tile, make the following changes:</p>
<ul>
<li>If the tile is <code>#</code>, the new map contains <code>##</code> instead.</li>
<li>If the tile is <code>O</code>, the new map contains <code>[]</code> instead.</li>
<li>If the tile is <code>.</code>, the new map contains <code>..</code> instead.</li>
<li>If the tile is <code>@</code>, the new map contains <code>@.</code> instead.</li>
</ul>
<p>This will produce a new warehouse map which is twice as wide and with wide boxes that are represented by <code>[]</code>. (The robot does not change size.)</p>
<p>The larger example from before would now look like this:</p>
<pre><code>####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##....[]@.....[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################
</code></pre>
<p>Because boxes are now twice as wide but the robot is still the same size and speed, boxes can be aligned such that they directly push two other boxes at once. For example, consider this situation:</p>
<pre><code>#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

&lt;vv&lt;&lt;^^&lt;&lt;^^
</code></pre>

<p>After appropriately resizing this map, the robot would push around these boxes as follows:</p>
<pre><code>Initial state:
##############
##......##..##
##..........##
##....[][]<em>@</em>.##
##....[]....##
##..........##
##############

Move &lt;:
##############
##......##..##
##..........##
##...[][]<em>@</em>..##
##....[]....##
##..........##
##############

Move v:
##############
##......##..##
##..........##
##...[][]...##
##....[].<em>@</em>..##
##..........##
##############

Move v:
##############
##......##..##
##..........##
##...[][]...##
##....[]....##
##.......<em>@</em>..##
##############

Move &lt;:
##############
##......##..##
##..........##
##...[][]...##
##....[]....##
##......<em>@</em>...##
##############

Move &lt;:
##############
##......##..##
##..........##
##...[][]...##
##....[]....##
##.....<em>@</em>....##
##############

Move ^:
##############
##......##..##
##...[][]...##
##....[]....##
##.....<em>@</em>....##
##..........##
##############

Move ^:
##############
##......##..##
##...[][]...##
##....[]....##
##.....<em>@</em>....##
##..........##
##############

Move &lt;:
##############
##......##..##
##...[][]...##
##....[]....##
##....<em>@</em>.....##
##..........##
##############

Move &lt;:
##############
##......##..##
##...[][]...##
##....[]....##
##...<em>@</em>......##
##..........##
##############

Move ^:
##############
##......##..##
##...[][]...##
##...<em>@</em>[]....##
##..........##
##..........##
##############

Move ^:
##############
##...[].##..##
##...<em>@</em>.[]...##
##....[]....##
##..........##
##..........##
##############
</code></pre>

<p>This warehouse also uses GPS to locate the boxes. For these larger boxes, distances are measured from the edge of the map to the closest edge of the box in question. So, the box shown below has a distance of <code>1</code> from the top edge of the map and <code>5</code> from the left edge of the map, resulting in a GPS coordinate of <code>100 * 1 + 5 = 105</code>.</p>
<pre><code>##########
##...[]...
##........
</code></pre>
<p>In the scaled-up version of the larger example from above, after the robot has finished all of its moves, the warehouse would look like this:</p>
<pre><code>####################
##[].......[].[][]##
##[]...........[].##
##[]........[][][]##
##[]......[]....[]##
##..##......[]....##
##..[]............##
##..<em>@</em>......[].[][]##
##......[][]..[]..##
####################
</code></pre>
<p>The sum of these boxes' GPS coordinates is <code><em>9021</em></code>.</p>
<p>Predict the motion of the robot and boxes in this new, scaled-up warehouse. <em>What is the sum of all boxes' final GPS coordinates?</em></p>
</article>


In [66]:
from more_itertools import minmax, one


tests = [
    {
        "name": "test I",
        "s": """
                #############
                #...........#
                #..O.O......#
                #..#.O......#
                #..OO.......#
                #...O@......#
                #...........#
                #############

                >>^^<vv<<v<^^^^
        """,
        "expected": 2046,
    },
    {
        "name": "test II",
        "s": """
                #############
                #...........#
                #..O.O......#
                #..O.O......#
                #..OO.......#
                #...O@......#
                #...........#
                #############

                >>^^<vv<<v<^^^^
        """,
        "expected": 1652,
    },
    {
        "name": "test III",
        "s": """
                #############
                #...........#
                #...O.......#
                #..O.O......#
                #..OO.......#
                #...O@......#
                #...........#
                #############

                >>^^<vv<<v<^^^^
        """,
        "expected": 1544,
    },
    {
        "name": "Small Example",
        "s": """
                ########
                #..O.O.#
                ##@.O..#
                #...O..#
                #.#.O..#
                #...O..#
                #......#
                ########

                <^^>>>vv<v>>v<<
        """,
        "expected": 1751,
    },
    {
        "name": "Small Example II",
        "s": """
                #######
                #...#.#
                #.....#
                #..OO@#
                #..O..#
                #.....#
                #######

                <vv<<^^<<^^
        """,
        "expected": 105 + 207 + 306,
    },
    {
        "name": "Example",
        "s": """
                ##########
                #..O..O.O#
                #......O.#
                #.OO..O.O#
                #..O@..O.#
                #O#..O...#
                #O..O..O.#
                #.OO.O.OO#
                #....O...#
                ##########

                <vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
                vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
                ><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
                <<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
                ^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
                ^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
                >^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
                <><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
                ^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
                v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^
        """,
        "expected": 9021,
    },
]


class DoubleWareHouse(WareHouse):
    BOX = "[]"

    def _parse(self, s) -> tuple[list[list[str]], str]:
        warehouse, moves = super()._parse(s)
        double_warehouse = []

        for row in warehouse:
            double_row = []
            for cell in row:
                if cell == super().BOX:
                    double_row.append(self.BOX[0])
                    double_row.append(self.BOX[1])
                elif cell == self.ROBOT:
                    double_row.append(self.ROBOT)
                    double_row.append(".")
                else:
                    double_row.append(cell)
                    double_row.append(cell)

            double_warehouse.append(double_row)

        return double_warehouse, moves

    def _move_box(
        self, r: int, c: int, dr: int, dc: int, rr: int, cc: int
    ) -> tuple[int, int]:
        # determine number of O's and swap back from last to first
        # if first cell after last is empty
        if dr == 0:
            return super()._move_box(r, c, dr, dc, rr, cc)

        # robot = r,c speed = dr
        # coliding point = rr, cc

        stack = [(r, {c})]

        while self._stack_of_boxes_continue(stack):
            row, cols = stack[-1]

            row1 = row + dr
            cols1 = set()

            for c1 in cols:
                if self.wharehouse[row1][c1 - 1] == self.BOX[0]:
                    cols1.add(c1 - 1)
                if (
                    self.wharehouse[row1][c1] in self.BOX
                    or self.wharehouse[row1][c1] == self.WALL
                ):
                    cols1.add(c1)
                if self.wharehouse[row1][c1 + 1] == self.BOX[1]:
                    cols1.add(c1 + 1)

            stack.append((row1, cols1))

        row, cols = stack[-1]
        if all(self._is_empty(row, c) for c in cols):
            # move stack
            stack.pop()

            while stack:
                row, cols = stack.pop()
                for c in cols:
                    self._swap(row, c, row + dr, c)

            return rr, cc

        return r, c

    def _stack_of_boxes_continue(self, stack: list[tuple[int, int, int]]) -> bool:
        row, cols = stack[-1]
        if not cols:
            return False

        if len(cols) == 1:  # start
            return self.wharehouse[row][one(cols)] == self.ROBOT

        if any(self.wharehouse[row][c] == "#" for c in cols):
            return False

        return any(self._is_box(row, c) for c in cols)


@test(tests=tests[:])
def partII_test(s: str) -> int:
    return DoubleWareHouse.solve(s, debug=False)


Test test I passed, for partII_test.
Test test II passed, for partII_test.
Test test III passed, for partII_test.
Test Small Example passed, for partII_test.
Test Small Example II passed, for partII_test.
Test Example passed, for partII_test.
Success


In [67]:
print(f"Part II: { (a:=DoubleWareHouse.solve(puzzle))}")

Part II: 1429013


<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1429013 </code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
